# Plan

## Target Table Structure
- `recipe_id`: Unique identifier for each recipe
- `title`: Name of the recipe
- `ingredients_raw`: Original ingredients list
- `instructions`: Recipe cooking instructions
- `heat_processed`: Boolean indicating if recipe involves heat processing
- `vegan`: Boolean indicating if recipe is vegan
- `vegetarian`: Boolean indicating if recipe is vegetarian
- `ingredients_processed`: JSON array of ingredients with heat processing status
- `cuisine_tags`: JSON array of cuisine classifications

## Implementation Steps

### Phase 1: Basic Processing
1. **[Data Preparation](#data-preparation)**
   - Load and clean the original CSV dataset
   - Create empty columns for new features

2. **[Heat Processing Detection](#heat-processing-detection)**
   - Use keyword matching to determine if recipes involve heat (bake, fry, roast, etc.)
   - Set `heat_processed` boolean flag for each recipe

3. **[Dietary Classification](#dietary-classification)**
   - Create dictionaries of non-vegetarian/non-vegan ingredients
   - Analyze ingredients to set `vegan` and `vegetarian` boolean flags

### Phase 2: Advanced Processing (LLM Required)
2. **[Ingredient Heat Processing Analysis](#ingredient-heat-processing-analysis)**
   - **Only for recipes with `heat_processed=True`**
   - Use LLM to analyze which specific ingredients are heat processed
   - Store as structured JSON in `ingredients_processed` column

2. **[Cuisine Classification](#cuisine-classification)**
   - Use LLM to determine cuisine types for all recipes
   - Store as ordered array in `cuisine_tags` column


### imports

In [ ]:
import pandas as pd

# Phase 1: Basic Processing

### Data Preparation
   - Load and clean the original CSV dataset
   - Delete image name column and ingredients, rename cleaned_ingredients to ingredients
   - Create empty columns for new features

read table, add columns

In [23]:
# Read the CSV
df = pd.read_csv('Kaggle_dataset.csv', header=0, names=['recipe_id', 'title', 'ingredients_raw', 'instructions', 'image_name', 'ingredients'])

# Delete image name column and ingredients, rename cleaned_ingredients to ingredients
df = df.drop(['image_name', 'ingredients'], axis=1)

# Create empty columns for new features
df['ingredients_processed'] = None
df['heat_processed'] = False
df['cuisine_tags'] = None
df['vegan'] = False
df['vegetarian'] = False

# Reorder columns
df = df[['recipe_id', 'title', 'ingredients_raw', 'ingredients_processed', 'instructions', 'heat_processed', 'cuisine_tags', 'vegan', 'vegetarian']]

df.head()

,recipe_id,title,ingredients_raw,ingredients_processed,instructions,heat_processed,cuisine_tags,vegan,vegetarian
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...",None,"Pat chicken dry with paper towels, season all ...",False,None,False,False
1,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",None,Preheat oven to 400°F and line a rimmed baking...,False,None,False,False
2,2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",None,Place a rack in middle of oven; preheat to 400...,False,None,False,False
3,3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",None,Preheat oven to 350°F with rack in middle. Gen...,False,None,False,False
4,4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",None,Stir together brown sugar and hot water in a c...,False,None,False,False


delete missing values

In [24]:
# list of columns to check - without ingredients_processed, cuisine_tags which are on purpose empty
columns_to_check = [col for col in df.columns if col not in ['ingredients_processed', 'cuisine_tags']]

# Show missing values in these columns before dropping
print("Missing values before dropping rows:")
print(df[columns_to_check].isnull().sum())

# Count total rows before dropping
rows_before = len(df)

# Drop rows with missing values in any of the specified columns
df = df.dropna(subset=columns_to_check)

# Count total rows after dropping
rows_after = len(df)

# Show how many rows were dropped
print(f"\nRows before: {rows_before}")
print(f"Rows after: {rows_after}")
print(f"Rows dropped: {rows_before - rows_after}")


Missing values before dropping rows:
recipe_id          0
title              5
ingredients_raw    0
instructions       8
heat_processed     0
vegan              0
vegetarian         0
dtype: int64

Rows before: 13501
Rows after: 13493
Rows dropped: 8


### Heat Processing Detection
   - Use keyword matching to determine if recipes involve heat (bake, fry, roast, etc.)
   - Set `heat_processed` boolean flag for each recipe

In [28]:
# Heat-related keywords
heat_keywords = ['bake', 'barbecue', 'blacken', 'blanch', 'blister', 'boil', 'braise', 
                'broil', 'brown', 'bubble', 'burn', 'caramelize', 'char', 'coddle', 
                'confit', 'convection', 'cook', 'crisp', 'crock pot', 'crust', 'deep-fry', 
                'deglaze', 'double-boil', 'fire', 'flame', 'flambé', 'flash-fry', 'fry', 
                'griddle', 'grill', 'hard-boil', 'heat', 'hot', 'hot-smoke', 'induction', 
                'infuse', 'melt', 'microwave', 'oven', 'pan-fry', 'pan-sear', 'parboil', 
                'poach', 'preheat', 'pressure cook', 'quick-broil', 'reduce', 'reheat', 
                'render', 'roast', 'rotisserie', 'salamander', 'sauté', 'scald', 'scorch', 
                'sear', 'shallow-fry', 'simmer', 'sizzle', 'skillet', 'slow cook', 'smoke', 
                'smoke-roast', 'soft-boil', 'sous vide', 'steam', 'steep', 'stew', 
                'stir-fry', 'sweat', 'temper', 'toast', 'torch', 'warm', 'water bath', 'wok']


# Check if heat kewords are in the instructions and update heat_processed column
df['heat_processed'] = df['instructions'].str.lower().apply(
    lambda x: any(keyword in x for keyword in heat_keywords)
)

df.head()


,recipe_id,title,ingredients_raw,ingredients_processed,instructions,heat_processed,cuisine_tags,vegan,vegetarian
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...",None,"Pat chicken dry with paper towels, season all ...",True,None,False,False
1,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",None,Preheat oven to 400°F and line a rimmed baking...,True,None,False,True
2,2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",None,Place a rack in middle of oven; preheat to 400...,True,None,False,True
3,3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",None,Preheat oven to 350°F with rack in middle. Gen...,True,None,False,False
4,4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",None,Stir together brown sugar and hot water in a c...,True,None,False,True


### Dietary Classification
   - Create dictionaries of non-vegetarian/non-vegan ingredients
   - Analyze ingredients to set `vegan` and `vegetarian` boolean flags

In [26]:
# Non-vegetarian ingredients
non_vegetarian_ingredients = ['meat', 'chicken', 'beef', 'pork', 'lamb', 'veal', 'turkey', 
                            'duck', 'goose', 'fish', 'salmon', 'tuna', 'shrimp', 'prawn',
                            'crab', 'lobster', 'oyster', 'mussel', 'clam', 'scallop',
                            'anchovy', 'bacon', 'ham', 'sausage', 'gelatin', 'lard',
                            'suet', 'stock', 'broth', 'venison', 'rabbit', 'quail', 'pheasant',
                            'bison', 'buffalo', 'elk', 'deer', 'squab', 'liver', 'kidney',
                            'heart', 'tongue', 'tripe', 'sweetbread', 'foie gras', 'caviar', 
                            'cod', 'halibut', 'tilapia', 'sardine', 'herring', 'mackerel',
                            'catfish', 'trout', 'flounder', 'mahi mahi', 'swordfish', 'eel',
                            'octopus', 'squid', 'calamari', 'chorizo', 'pepperoni', 'salami',
                            'prosciutto', 'pancetta', 'bologna', 'hot dog', 'jerky', 'pate',
                            'bone marrow', 'bone broth', 'animal fat', 'tallow', 'schmaltz',
                            'collagen', 'isinglass', 'rennet', 'animal shortening', 'cochineal',
                            'carmine', 'shellac', 'confectioner\'s glaze', 'omega-3', 'fish sauce',
                            'worcestershire sauce', 'caesar dressing', 'dashi', 'katsuobushi',
                            'bonito', 'escargot', 'frog legs']

# Additional non-vegan ingredients (on top of non-vegetarian ones)
non_vegan_ingredients = ['egg', 'milk', 'cream', 'butter', 'cheese', 'yogurt', 'honey',
                        'mayonnaise', 'whey', 'casein', 'ghee', 'lactose', 'rennet',
                        'albumin', 'carmine', 'shellac', 'royal jelly', 'beeswax', 'propolis',
                        'bee pollen', 'buttermilk', 'kefir', 'sour cream', 'ice cream',
                        'custard', 'pudding', 'creme fraiche', 'mascarpone', 'ricotta',
                        'cottage cheese', 'quark', 'paneer', 'egg white', 'egg yolk',
                        'meringue', 'hollandaise', 'marshmallow', 'frosting', 'nougat',
                        'whipped cream', 'condensed milk', 'evaporated milk', 'powdered milk',
                        'milk solids', 'milk protein', 'lactose', 'caseinates', 'lactoferrin',
                        'lactitol', 'lactoglobulin', 'lactalbumin', 'recaldent', 'curds',
                        'vitamin D3', 'lanolin', 'pepsin', 'trypsin', 'glycerin', 'glycerol',
                        'stearic acid', 'oleic acid', 'capric acid', 'myristic acid',
                        'palmitic acid', 'l-cysteine', 'keratin', 'elastin', 'cetyl alcohol',
                        'cholesterol', 'lecithin', 'mono and diglycerides', 'natural flavor',
                        'e120', 'e441', 'e542', 'e631', 'e901', 'e904', 'e910', 'e920',
                        'e921', 'e966', 'e1105']


# Check if ingredients contain non-vegetarian/non-vegan items and update columns
df['vegetarian'] = df['ingredients_raw'].str.lower().apply(
    lambda x: not any(ingredient in str(x).lower() for ingredient in non_vegetarian_ingredients)
)

df['vegan'] = df['ingredients_raw'].str.lower().apply(
    lambda x: not any(ingredient in str(x).lower() for ingredient in non_vegetarian_ingredients + non_vegan_ingredients)
)

df.head()


,recipe_id,title,ingredients_raw,ingredients_processed,instructions,heat_processed,cuisine_tags,vegan,vegetarian
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...",None,"Pat chicken dry with paper towels, season all ...",True,None,False,False
1,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",None,Preheat oven to 400°F and line a rimmed baking...,True,None,False,True
2,2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",None,Place a rack in middle of oven; preheat to 400...,True,None,False,True
3,3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",None,Preheat oven to 350°F with rack in middle. Gen...,True,None,False,False
4,4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",None,Stir together brown sugar and hot water in a c...,True,None,False,True


# Phase 2: Advanced Processing (LLM Required)

### Ingredient Heat Processing Analysis
   - **Only for recipes with `heat_processed=True`**
   - Use LLM to analyze which specific ingredients are heat processed
   - Store as structured JSON in `ingredients_processed` column

### Cuisine Classification
   - Use LLM to determine cuisine types for all recipes
   - Store as ordered array in `cuisine_tags` column